<a href="https://colab.research.google.com/github/Karthick47v2/mcq-generator/blob/base-dev/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade simplet5
! pip install fastt5

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from simplet5 import SimpleT5
from fastT5 import generate_onnx_representation, quantize

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [3]:
df = pd.read_csv('gdrive/MyDrive/mcq-gen/SQuAD-processed.csv')

In [ ]:
df.head()

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2)
train_df.shape, test_df.shape

In [7]:
model = SimpleT5()
model.from_pretrained(model_type='t5', model_name='t5-base')

In [8]:
model.train(train_df=train_df[:1000], eval_df=test_df[:200], source_max_token_len=512, target_max_token_len=96, batch_size=8, max_epochs=3, use_gpu=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: /content/lightning_logs

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [22]:
model_dir = 'outputs/simplet5-epoch-1-train-loss-1.4667-val-loss-1.5138'

In [23]:
model.load_model(model_type='t5', model_dir=model_dir, use_gpu=True)

In [13]:
text = "context: President Donald Trump said and predicted that some states would reopen this month. answer: Donald Trump"
model.predict(source_text=text, num_return_sequences=3, num_beams=5, max_length=72)

['Who predicted that some states would reopen this month?',
 'Who predicted some states would reopen this month?',
 'Who said that some states would reopen this month?']

In [25]:
onnx_model = generate_onnx_representation(pretrained_version=model_dir, input_sequence_length=512)

Exporting to onnx... |################################| 3/3


In [27]:
quantized_model = quantize(onnx_model)

Quantizing... |################################| 3/3
